一、项目背景
-
通过飞桨提供的高层API，调用飞桨封装好的ResNet模型。学习ResNet模型的特点

二、数据介绍
-
使用Cifar10数据集,CIFAR-10数据集由10个类中的60000个32x32彩色图像组成，每个类有6000个图像。有50000个训练图像和10000个测试图像。

三、模型介绍
-
ResNet是2015年ImageNet比赛的冠军，将识别错误率降低到了3.6%，这个结果甚至超出了正常人眼识别的精度。
随着深度学习的不断发展，模型的层数越来越多，网络结构也越来越复杂。那么是否加深网络结构，就一定会得到更好的效果呢？从理论上来说，假设新增加的层都是恒等映射，只要原有的层学出跟原模型一样的参数，那么深模型结构就能达到原模型结构的效果。换句话说，原模型的解只是新模型的解的子空间，在新模型解的空间里应该能找到比原模型解对应的子空间更好的结果。但是实践表明，增加网络的层数之后，训练误差往往不降反升。

Kaiming He等人提出了残差网络ResNet来解决上述问题，其基本思想如 图6所示。

图1(a)：表示增加网络的时候，将xxx映射成y=F(x)y=F(x)y=F(x)输出。
图1(b)：对图1(a)作了改进，输出y=F(x)+xy=F(x) + xy=F(x)+x。这时不是直接学习输出特征yyy的表示，而是学习y−xy-xy−x。
如果想学习出原模型的表示，只需将F(x)F(x)F(x)的参数全部设置为0，则y=xy=xy=x是恒等映射。
F(x)=y−xF(x) = y - xF(x)=y−x也叫做残差项，如果x→yx\rightarrow yx→y的映射接近恒等映射，图1(b)中通过学习残差项也比图6(a)学习完整映射形式更加容易。
![](https://ai-studio-static-online.cdn.bcebos.com/b8a83393f2a64c28a23718b7f1246981efdf1b7f949a4cad8dff008a2e02ce04)


                                           图1：残差设计思想

图1(b)的结构是残差网络的基础，这种结构也叫做残差块（Residual block）。输入xxx通过跨层连接，能更快的向前传播数据，或者向后传播梯度。通俗的比喻，在火热的电视节目《王牌对王牌》上有一个“传声筒”的游戏，排在队首的嘉宾把看到的影视片段表演给后面一个嘉宾看，经过四五个嘉宾后，最后一个嘉宾如果能表演出更多原剧的内容，就能取得高分。我们常常会发现刚开始的嘉宾往往表演出最多的信息（类似于Loss），而随着表演的传递，有效的表演信息越来越少（类似于梯度弥散）。如果每个嘉宾都能看到原始的影视片段，那么相信传声筒的效果会好很多。类似的，由于ResNet每层都存在直连的旁路，相当于每一层都和最终的损失有“直接对话”的机会，自然可以更好的解决梯度弥散的问题。残差块的具体设计方案如 图2 所示，这种设计方案也常称作瓶颈结构（BottleNeck）。1*1的卷积核可以非常方便的调整中间层的通道数，在进入3*3的卷积层之前减少通道数（256->64），经过该卷积层后再恢复通道数(64->256)，可以显著减少网络的参数量。这个结构（256->64->256）像一个中间细，两头粗的瓶颈，所以被称为“BottleNeck”。

![](https://ai-studio-static-online.cdn.bcebos.com/51b4659e4e2d42d6bc085b742f58d317857b63937e244f5882c815e7055d09cf)
                                     
                                            图2：残差块结构示意图
                     
下图表示出了ResNet-50的结构，一共包含49层卷积和1层全连接，所以被称为ResNet-50。
![](https://ai-studio-static-online.cdn.bcebos.com/ff22fb48e71a471f93a12e30df5271faaec2c0e8a73c4f5c9b141b43a97ab698)

                                        图3：ResNet-50模型网络结构示意图



四、程序实现
-


In [1]:
# 从paddle.vision.models 模块中import 残差网络，VGG网络，LeNet网络
import paddle
from paddle.vision.models import resnet50, vgg16, LeNet
from paddle.vision.datasets import Cifar10
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.transforms import Transpose

In [4]:
import numpy as np
import matplotlib.pyplot as plt


# 使用Cifar10数据集
train_dataset = Cifar10(mode='train', transform=Transpose())
val_dataset = Cifar10(mode='test', transform=Transpose())




In [5]:
# 确保从paddle.vision.datasets.Cifar10中加载的图像数据是np.ndarray类型
paddle.vision.set_image_backend('cv2')
# 调用resnet50模型
model = paddle.Model(resnet50(pretrained=False, num_classes=10))


# 定义优化器
optimizer = Momentum(learning_rate=0.01,
                     momentum=0.9,
                     weight_decay=L2Decay(1e-4),
                     parameters=model.parameters())
# 进行训练前准备
model.prepare(optimizer, CrossEntropyLoss(), Accuracy(topk=(1, 5)))
# 启动训练
model.fit(train_dataset,
          val_dataset,
          epochs=50,
          batch_size=64,
          save_dir="./output",
          num_workers=8)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step  10/782 - loss: 5.6795 - acc_top1: 0.1344 - acc_top5: 0.5219 - 152ms/step
step  20/782 - loss: 4.6656 - acc_top1: 0.1562 - acc_top5: 0.5680 - 95ms/step
step  30/782 - loss: 3.3192 - acc_top1: 0.1609 - acc_top5: 0.5901 - 78ms/step
step  40/782 - loss: 2.9309 - acc_top1: 0.1707 - acc_top5: 0.6145 - 73ms/step
step  50/782 - loss: 3.4723 - acc_top1: 0.1741 - acc_top5: 0.6297 - 71ms/step
step  60/782 - loss: 3.3919 - acc_top1: 0.1773 - acc_top5: 0.6388 - 70ms/step
step  70/782 - loss: 5.8369 - acc_top1: 0.1837 - acc_top5: 0.6507 - 68ms/step
step  80/782 - loss: 8.3766 - acc_top1: 0.1893 - acc_top5: 0.6553 - 64ms/step
step  90/782 - loss: 2.6302 - acc_top1: 0.1910 - acc_top5: 0.6585 - 62ms/step
step 100/782 - loss: 2.5693 - acc_top1: 0.1889 - acc_top5: 0.6573 - 59ms/step
step 110/782 - loss: 5.8991 - acc_top1: 0.1884 - acc_top5: 0.6585 - 58ms/step
step 120/782 - loss: 3.0208 - acc_top1: 0.1904 - acc_top5: 0.6641 - 56ms/step
step 130/782 - loss: 4.7586 - acc_top1: 0.1906 - acc_top5: 0.66

KeyboardInterrupt: 

In [ ]:
# 模型评估
model.evaluate(val_dataset, batch_size=64, verbose=1)

In [ ]:
# 保存模型参数
model.save('RESNEGT50')  # save for training
model.save('RESNEGT50', False)  # save for inference